In [1]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
# today = (datetime.today()-timedelta(days=1)).strftime("%Y%m%d") #오늘
today = datetime.today().strftime("%Y%m%d") #오늘
# s_today = '20241223'
# print(today)
dart_list = dart.list(start=today,end=today)
lists = dart_list[dart_list['report_nm'].str.contains('공급계약체결')&~dart_list['report_nm'].str.contains('기재정정')]
print(f'{len(lists)}개 조회')
def supply(rcp_no,corp):
    
    
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt[[0,1,2]]
    tt[2] = tt.apply(lambda row: row[1] if pd.isna(row[2]) else row[2], axis=1)
    tt = tt[[1,2]]
    tt = tt.T
    
    if '내용' in tt.iloc[0,0]: # 공급계약은 두가지 타입으로 존재
        tt = tt[[20,0,2,5,6,7,12,13,14,21,20]]
        
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()
        tt = tt.drop(columns='index')
    else:
        tt = tt[[13,1,2,3,4,6,8,9,10,14,15]]
        # print(tt)
        tt.columns = ['계약일자','계약내용','계약금액(원)','최근매출액(원)','매출액대비(%)','계약상대','판매공급지역','시작일','종료일','유보사유','유보기한']
        tt = tt.drop(tt.index[0]).reset_index()

        tt = tt.drop(columns='index')
    tt.insert(0,'기업',corp)
    time.sleep(1)
    return tt


df = pd.DataFrame()

for i in range(len(lists)):
    try:
        rcp_no = lists['rcept_no'].values[i]
        corp = lists['flr_nm'].values[i]
        supply_df = supply(rcp_no,corp)
        print(f'{i}_{corp} 완료')  
        df = pd.concat([df,supply_df])
    except Exception as e:
        print(f'{corp}',e)
        
file_name = f'C:\투자기록\정리\코딩\장내매수\공급계약_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\codek\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\codek\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\codek\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config

AttributeError: _ARRAY_API not found

20250119
142개 조회
0_소룩스 완료
1_파두 완료
2_유바이오로직스 완료
3_씨에스윈드 완료
4_디지틀조선 완료
5_엔텔스 완료
6_딥노이드 완료
7_네이블 완료
8_빅텍 완료
9_SCL사이언스 완료
10_청호ICT 완료
11_진흥기업 완료
12_한미반도체 완료
13_이노시뮬레이션 완료
14_삼성바이오로직스 완료
15_상지건설 완료
16_이수스페셜티케미컬 완료
17_엠오티 완료
18_세명전기 완료
19_디아이티 완료
20_쏘카 완료
효성 "None of [Index([13, 1, 2, 3, 4, 6, 8, 9, 10, 14, 15], dtype='int64')] are in the [columns]"
22_효성중공업 완료
23_GS건설 완료
24_제룡산업 완료
25_산일전기 완료
26_삼호개발 완료
27_삼성E&A 완료
28_아이씨디 완료
29_한솔피엔에스 완료
30_웨이브일렉트로 완료
HL홀딩스 "None of [Index([13, 1, 2, 3, 4, 6, 8, 9, 10, 14, 15], dtype='int64')] are in the [columns]"
32_HL D&I 완료
33_포스코엠텍 완료
34_쇼박스 완료
35_엔지스테크널러지 완료
36_피노 완료
37_퀄리타스반도체 완료
38_NPX 완료
코오롱 "None of [Index([13, 1, 2, 3, 4, 6, 8, 9, 10, 14, 15], dtype='int64')] are in the [columns]"
40_코오롱글로벌 완료
41_한화엔진 완료
42_KTcs 완료
43_HS화성 완료
44_케이엔알시스템 완료
45_SCL사이언스 완료
46_우진아이엔에스 완료
47_한화엔진 완료
48_HS화성 완료
49_우진아이엔에스 완료
50_테스 완료
51_피노 완료
52_이엔셀 완료
53_삼호개발 완료
54_DB 완료
55_DB 완료
56_DB 완료
57_드림씨아이에스 완료
58_컨텍 완료
59_범한퓨얼셀 완료
60_파인텍 완료
효성 "None of [Index([13, 1, 2, 3, 4